In [1]:
!pip install -q transformers accelerate datasets bitsandbytes


In [2]:
!apt update
!apt install -y libmariadb-dev

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 128 kB in 1s (85.7 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
38 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'http

In [3]:
!pip install mysql-connector-python sqlalchemy mariadb

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!git clone https://github.com/Horizontal-Labs/Argument-Mining.git

fatal: destination path 'Argument-Mining' already exists and is not an empty directory.


In [6]:
# change direcotry
import sys
sys.path.append('/content/Argument-Mining')

In [7]:
! pip install -U "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 30.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.32.3
    Uninstalling huggingface-hub-0.32.3:
      Successfully uninstalled huggingface-hub-0.32.3


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import random
import pandas as pd
import torch
import os
from huggingface_hub import login
login("your_hf_token")


In [9]:
model_id = "deepseek-ai/deepseek-llm-7b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # automatically assigns to available GPU/CPU
    torch_dtype=torch.bfloat16,  # or float16
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [10]:
from db.db import get_engine, get_session, get_db_session

# Get a session
session = get_session()

In [11]:

try:
    from db.queries import get_test_data
    claims, premises, labels = get_test_data()
    data = [
        {"claim": c.text, "premise": p.text, "label": l}
        for c, p, l in zip(claims[:100], premises[:100], labels[:100])
    ]
except Exception as e:
    print("Failed to load test data from database. Falling back to mock data.")
    print("Error:", e)
    data = [
        {
            "claim": "Social media has a negative impact on mental health.",
            "premise": "Studies show increased anxiety and depression among teens using social platforms.",
            "label": "stance_pro"
        },
        {
            "claim": "School uniforms should be mandatory.",
            "premise": "Uniforms limit students' freedom of expression and individuality.",
            "label": "stance_con"
        }
    ] * 50



In [65]:
#A_FEW_SHOT_PROMPT works best
A_FEW_SHOT_PROMPT = lambda claim, premise: f"""
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:
Claim: Climate change is primarily caused by human activity.
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to human activities like burning fossil fuels and deforestation.
Stance: Supports

Example 6:
Claim: Video games can improve cognitive skills.
Evidence: Research indicates that playing strategy games can improve memory, attention, and problem-solving skills.
Stance: Supports

Example 7:
Claim: Exercise has no effect on weight loss.
Evidence: Scientific research consistently shows that physical activity increases energy expenditure and supports weight management.
Stance: Refutes

Example 8:
Claim: Social media has no impact on mental health.
Evidence: Multiple studies have linked social media usage with increased rates of anxiety and depression in teenagers and young adults.
Stance: Refutes


Now analyze the following:

Claim: {claim}
Evidence: {premise}
Stance:"""

B_FEW_SHOT_PROMPT = lambda claim, premise: f"""
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.

Example 1:
Claim: Climate change is primarily caused by human activity.
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to human activities like burning fossil fuels and deforestation.
Stance: Supports

Example 2:
Claim: Video games can improve cognitive skills.
Evidence: Research indicates that playing strategy games can improve memory, attention, and problem-solving skills.
Stance: Supports

Example 3:
Claim: Exercise has no effect on weight loss.
Evidence: Scientific research consistently shows that physical activity increases energy expenditure and supports weight management.
Stance: Refutes

Example 4:
Claim: Social media has no impact on mental health.
Evidence: Multiple studies have linked social media usage with increased rates of anxiety and depression in teenagers and young adults.
Stance: Refutes

Example 5:
Claim: Electric cars are better for the environment than gasoline cars.
Evidence: Electric cars emit significantly fewer greenhouse gases compared to gasoline cars, even when accounting for electricity production.
Stance: Supports

Example 6:
Claim: The Earth is flat.
Evidence: Satellite imagery and circumnavigation of the globe confirm that the Earth is round.
Stance: Refutes

Example 7:
Claim: All processed foods are healthy.
Evidence: Nutrition experts warn that processed foods often contain added sugars, unhealthy fats, and preservatives.
Stance: Refutes

Example 8:
Claim: Exercising regularly improves mental health.
Evidence: Several clinical studies have found that consistent exercise helps reduce symptoms of depression and anxiety.
Stance: Supports

Example 9:
Claim: Vaccines are effective in preventing diseases.
Evidence: Data from the CDC shows that vaccination reduces the risk of infection by up to 95% in most cases.
Stance: Supports

Example 10:
Claim: Smoking is harmless to health.
Evidence: The World Health Organization states that smoking is a leading cause of cancer, respiratory diseases, and cardiovascular issues.
Stance: Refutes

Now, analyse the following and reply with only one word: either "Supports" or "Refutes" accordingly:
Claim: {claim}
Evidence: {premise}
Stance:"""

C_FEW_SHOT_PROMPT = lambda claim, premise: f"""
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Stance: Refutes

Example 5:
Claim: Climate change is primarily caused by human activity.
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to human activities like burning fossil fuels and deforestation.
Stance: Supports

Example 6:
Claim: Video games can improve cognitive skills.
Evidence: Research indicates that playing strategy games can improve memory, attention, and problem-solving skills.
Stance: Supports

Example 7:
Claim: Exercise has no effect on weight loss.
Evidence: Scientific research consistently shows that physical activity increases energy expenditure and supports weight management.
Stance: Refutes

Example 8:
Claim: Social media has no impact on mental health.
Evidence: Multiple studies have linked social media usage with increased rates of anxiety and depression in teenagers and young adults.
Stance: Refutes


Now analyze the following and provide only one word answer: "Supports" or "Refutes". No Explaination.:

Claim: {claim}
Evidence: {premise}
Stance:"""

D_FEW_SHOT_PROMPT = lambda claim, premise: f"""
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".
Explain your reasoning briefly for each example before giving your final stance.

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Reasoning: The evidence scientifically explains why the sky is blue.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Reasoning: The evidence shows that fast food is linked to health problems, contradicting the claim.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Reasoning: The evidence directly supports the claim that solar energy cuts electricity costs.
Stance: Supports

Example 4:
Claim: Exercise is harmful to health.
Evidence: Numerous studies show that regular exercise improves cardiovascular health and longevity.
Reasoning: The evidence contradicts the claim by showing positive health effects of exercise.
Stance: Refutes

Example 5:
Claim: Climate change is primarily caused by human activity.
Evidence: Recent studies show that over 90% of the increase in greenhouse gases is due to human activities like burning fossil fuels and deforestation.
Reasoning: The evidence links human activity directly to the rise in greenhouse gases, supporting the claim.
Stance: Supports

Example 6:
Claim: Video games can improve cognitive skills.
Evidence: Research indicates that playing strategy games can improve memory, attention, and problem-solving skills.
Reasoning: The evidence describes cognitive benefits from playing strategy games, supporting the claim.
Stance: Supports

Example 7:
Claim: Exercise has no effect on weight loss.
Evidence: Scientific research consistently shows that physical activity increases energy expenditure and supports weight management.
Reasoning: The evidence shows that exercise contributes to weight management, refuting the claim.
Stance: Refutes

Example 8:
Claim: Social media has no impact on mental health.
Evidence: Multiple studies have linked social media usage with increased rates of anxiety and depression in teenagers and young adults.
Reasoning: The evidence directly contradicts the claim by showing mental health impacts of social media.
Stance: Refutes

Now analyze the following:

Claim: {claim}
Evidence: {premise}
Reasoning:
Stance:
"""




In [20]:
ZEROSHOT_PROMPT = lambda claim, premise: f"""
Claim: {claim}
Evidence: {premise}
Given the Claim and evidence, does the evidence support or refute the claim? Respond with one word accordingly: "Supports" or "Refutes".
"""

In [55]:

import re

def normalize_output(output: str) -> str:
    output = output.strip().lower()

    # Find all stance lines
    stance_lines = re.findall(r"stance\s*[:\-]?\s*(supports|refutes)", output)

    if stance_lines:
        # Take the LAST one, since it's the final response
        return stance_lines[-1]

    # Fallback: try to get first matching standalone word
    for word in output.split():
        if "supports" in word:
            return "supports"
        elif "refutes" in word:
            return "refutes"

    return "unknown"



In [56]:
reverse_label_map = {
    "supports": "stance_pro",
    "refutes": "stance_con"
}


In [66]:
results = []
MODE = "few-shot"  # or "zero-shot"

for example in data:
    claim = example["claim"]
    premise = example["premise"]
    gold = example["label"]


    prompt = D_FEW_SHOT_PROMPT(claim, premise)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=10, do_sample=False, temperature=0.0, top_p=0.9)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Normalization & mapping
    normalized_prediction = normalize_output(decoded)  # e.g. "Supports", "Refutes"
    mapped_prediction = reverse_label_map.get(normalized_prediction, "unknown")  # stance_pro, stance_con, unknown

    # DEBUG print
    first_token = decoded.lower().strip().split()[0].strip(".,:")
    print(f"🔍 Raw: {decoded} → Token: {first_token} → Label: {normalized_prediction}")

    # Store result
    results.append({
        "claim": claim,
        "premise": premise,
        "true_label": gold,
        "model_output": decoded.strip(),
        "normalized_prediction": normalized_prediction,
        "mapped_prediction": mapped_prediction,
        "correct": mapped_prediction == gold
    })

print("✅ All examples processed!")


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


🔍 Raw: 
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".
Explain your reasoning briefly for each example before giving your final stance.

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Reasoning: The evidence scientifically explains why the sky is blue.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Reasoning: The evidence shows that fast food is linked to health problems, contradicting the claim.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Reasoning: The evidence directly supports the claim that solar energy cuts electricity costs.
Stanc

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


🔍 Raw: 
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".
Explain your reasoning briefly for each example before giving your final stance.

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Reasoning: The evidence scientifically explains why the sky is blue.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Reasoning: The evidence shows that fast food is linked to health problems, contradicting the claim.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Reasoning: The evidence directly supports the claim that solar energy cuts electricity costs.
Stanc

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


...

🔍 Raw: 
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".
Explain your reasoning briefly for each example before giving your final stance.

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Reasoning: The evidence scientifically explains why the sky is blue.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Reasoning: The evidence shows that fast food is linked to health problems, contradicting the claim.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Reasoning: The evidence directly supports the claim that solar energy cuts electricity costs.
Stanc

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


🔍 Raw: 
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".
Explain your reasoning briefly for each example before giving your final stance.

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Reasoning: The evidence scientifically explains why the sky is blue.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Reasoning: The evidence shows that fast food is linked to health problems, contradicting the claim.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Reasoning: The evidence directly supports the claim that solar energy cuts electricity costs.
Stanc

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


🔍 Raw: 
You are given a claim and evidence. Your task is to determine whether the evidence supports or refutes the claim.
Respond only with one word: "Supports" or "Refutes".
Explain your reasoning briefly for each example before giving your final stance.

Example 1:
Claim: The sky is blue.
Evidence: Scientists have confirmed that the sky appears blue due to the scattering of light.
Reasoning: The evidence scientifically explains why the sky is blue.
Stance: Supports

Example 2:
Claim: Eating fast food is healthy.
Evidence: Studies show that fast food contains high levels of trans fats and sodium, leading to health problems.
Reasoning: The evidence shows that fast food is linked to health problems, contradicting the claim.
Stance: Refutes

Example 3:
Claim: Solar energy reduces electricity costs.
Evidence: Households using solar panels report a significant reduction in electricity bills.
Reasoning: The evidence directly supports the claim that solar energy cuts electricity costs.
Stanc

In [67]:
print(f"🔍 Normalized: {normalized_prediction} → Mapped label: {mapped_prediction}")


🔍 Normalized: refutes → Mapped label: stance_con


In [68]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

MODEL_NAME = model_id
results_df = pd.DataFrame(results)

'''# Optional: drop "unknown" (NotEnoughInfo) predictions
filtered_df = results_df[results_df["mapped_prediction"] != "unknown"]'''
y_true = results_df["true_label"]
y_pred = results_df["mapped_prediction"]

print(f"\nFinal Accuracy ({MODE} using {MODEL_NAME}): {accuracy_score(y_true, y_pred):.2%}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, labels=["stance_pro", "stance_con"]))
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred, labels=["stance_pro", "stance_con"]))

acc = results_df["correct"].mean()
print(acc)



Final Accuracy (few-shot using deepseek-ai/deepseek-llm-7b-chat): 54.00%

Classification Report:
              precision    recall  f1-score   support

  stance_pro       0.71      0.10      0.18        49
  stance_con       0.53      0.96      0.68        51

    accuracy                           0.54       100
   macro avg       0.62      0.53      0.43       100
weighted avg       0.62      0.54      0.43       100


Confusion Matrix:
[[ 5 44]
 [ 2 49]]
0.54


In [45]:
import pandas as pd
import os

# Assuming 'results' is your final list of dictionaries
# Convert directly to DataFrame
s_df = pd.DataFrame(results)

# Columns to keep
columns_to_export = ["claim", "premise", "true_label","normalized_prediction", "mapped_prediction", "model_output", "correct"]

# Check if file already exists
s_file = "stance_predictions_summary.csv"
if os.path.exists(s_file):
    # Append only the selected columns to the existing CSV
    s_df[columns_to_export].to_csv(s_file, mode='a', header=False, index=False)
else:
    # Create new CSV with the header
    s_df[columns_to_export].to_csv(s_file, index=False)

print("✅ CSV with selected columns saved as 'stance_predictions_summary.csv'!")


✅ CSV with selected columns saved as 'stance_predictions_summary.csv'!


In [69]:
from datetime import datetime
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

if "experiment_log" not in globals():
    experiment_log = pd.DataFrame(columns=[
        "timestamp", "model", "mode", "accuracy",
        "precision_pro", "recall_pro", "f1_pro",
        "precision_con", "recall_con", "f1_con"
    ])



report = classification_report(y_true, y_pred, labels=["stance_pro", "stance_con"], output_dict=True)

summary = {
    "timestamp": datetime.now().isoformat(timespec='seconds'),
    "model": MODEL_NAME,
    "mode": MODE,
    "accuracy": accuracy_score(y_true, y_pred),
    "precision_pro": precision_score(y_true, y_pred, pos_label="stance_pro", zero_division=0),
    "recall_pro": recall_score(y_true, y_pred, pos_label="stance_pro", zero_division=0),
    "f1_pro": f1_score(y_true, y_pred, pos_label="stance_pro", zero_division=0),
    "precision_con": precision_score(y_true, y_pred, pos_label="stance_con", zero_division=0),
    "recall_con": recall_score(y_true, y_pred, pos_label="stance_con", zero_division=0),
    "f1_con": f1_score(y_true, y_pred, pos_label="stance_con", zero_division=0),
}

experiment_log = pd.DataFrame(columns=summary.keys())
# Append and print updated table
experiment_log = pd.concat([experiment_log, pd.DataFrame([summary])], ignore_index=True)
print("📊 Summary of All Experiments So Far:")
display(experiment_log)


📊 Summary of All Experiments So Far:


<ipython-input-69-4ee94a84f8af>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_log = pd.concat([experiment_log, pd.DataFrame([summary])], ignore_index=True)


,timestamp,model,mode,accuracy,precision_pro,recall_pro,f1_pro,precision_con,recall_con,f1_con
0,2025-06-03T11:10:49,deepseek-ai/deepseek-llm-7b-chat,few-shot,0.54,0.714286,0.102041,0.178571,0.526882,0.960784,0.680556


In [70]:
summary_file = "stance_eval_summary.csv"
summary_df = pd.DataFrame([summary])

# Append to CSV (create if it doesn't exist)
if os.path.exists(summary_file):
    summary_df.to_csv(summary_file, mode='a', header=False, index=False)
else:
    summary_df.to_csv(summary_file, index=False)


In [71]:
if os.path.exists(summary_file):
    print("\n📊 Summary of All Experiments So Far:")
    display(pd.read_csv(summary_file))



📊 Summary of All Experiments So Far:


,timestamp,model,mode,accuracy,precision_pro,recall_pro,f1_pro,precision_con,recall_con,f1_con
0,2025-06-03T09:12:48,TinyLlama/TinyLlama-1.1B-Chat-v1.0,few-shot,0.491,0.489011,0.355289,0.411561,0.492138,0.627255,0.551542
1,2025-06-03T10:50:22,deepseek-ai/deepseek-llm-7b-chat,zero-shot,0.650,0.888889,0.326531,0.477612,0.597561,0.960784,0.736842
2,2025-06-03T10:50:22,deepseek-ai/deepseek-llm-7b-chat,zero-shot,0.650,0.888889,0.326531,0.477612,0.597561,0.960784,0.736842
3,2025-06-03T11:10:49,deepseek-ai/deepseek-llm-7b-chat,few-shot,0.540,0.714286,0.102041,0.178571,0.526882,0.960784,0.680556
